In [ ]:
import numpy as np
import pyairsea

%matplotlib widget
import matplotlib.pyplot

# Transfer coefficients of heat and momentum

Compare with Fig 7a-c in
[Kondo, J. (1975). Air-sea bulk transfer coefficients in diabatic conditions. Boundary-Layer Meteorology, 9(1), 91–112](https://doi.org/10.1007/BF00232256)

In [ ]:
fig, (ax1, ax2, ax3) = matplotlib.pyplot.subplots(nrows=3, figsize=(10, 10))

w = np.linspace(0, 30.0, 100)
w.shape = -1, 1
Tw = np.full_like(w, 10.0 + 273.15)
for delta_t in [-5.0, -2.0, -1.0, 0.0, 1.0, 2.0, 5.0]:
    cd_mom, cd_sensible, cd_latent = (
        np.empty_like(w),
        np.empty_like(w),
        np.empty_like(w),
    )
    pyairsea.transfer_coefficients(
        1, Tw, Tw + delta_t, w, cd_mom, cd_sensible, cd_latent
    )
    ax1.plot(w, cd_mom, label="Tw-Ta=%s" % delta_t)
    ax2.plot(w, cd_sensible, label="Tw-Ta=%s" % delta_t)
    ax3.plot(w, cd_latent, label="Tw-Ta=%s" % delta_t)
for ax in (ax1, ax2, ax3):
    ax.grid()
    ax.set_xlabel("wind speed (m s-1)")
    ax.legend()
ax1.set_ylabel("C_DD (momentum)")
ax2.set_ylabel("C_HD (sensible)")
ax3.set_ylabel("C_ED (latent)")

# Longwave radiation

In [ ]:
T = np.linspace(-2.0, 35.0, 100)
T.shape = 1, -1
tw = ta = T
cloud = np.full_like(T, 0.9)
lat = np.full_like(T, 50.0)
relhum = np.full_like(T, 10.0)
sp = np.full_like(T, 101325.0)
es, ea, qs, qa, rhoa, ql = (
    np.empty_like(T),
    np.empty_like(T),
    np.empty_like(T),
    np.empty_like(T),
    np.empty_like(T),
    np.empty_like(T),
)
fig, ax = matplotlib.pyplot.subplots()
for method in pyairsea.LongwaveMethod:
    pyairsea.humidity(
        pyairsea.HumidityMeasure.RELATIVE_HUMIDITY,
        relhum,
        sp,
        tw,
        ta,
        es,
        ea,
        qs,
        qa,
        rhoa,
    )
    pyairsea.longwave_radiation(
        method, lat, tw + 273.15, ta + 273.15, cloud, ea, qa, ql
    )
    ax.plot(T[0, :], ql[0, :], label=method.name, alpha=0.5)
ax.set_xlabel("air/water temperature (degrees Celsius)")
ax.set_ylabel("net downward longwave flux (W m-2)")
ax.legend()
ax.grid()

# Air density

In [ ]:
T = np.linspace(-2.0, 35.0, 100)
T.shape = 1, -1
airp = np.full_like(T, 101325.0)  # 1 atm
es = np.empty_like(T)
ea = np.empty_like(T)
qs = np.empty_like(T)
qa = np.empty_like(T)
rhoa = np.empty_like(T)
fig, ax = matplotlib.pyplot.subplots()
for rh in (0.0, 25.0, 50.0, 75.0, 100.0):
    pyairsea.humidity(
        pyairsea.HumidityMeasure.RELATIVE_HUMIDITY,
        np.full_like(T, rh),
        airp,
        T,
        T,
        es,
        ea,
        qs,
        qa,
        rhoa,
    )
    ax.plot(T[0, :], rhoa[0, :], label="RH = %s %%" % rh, alpha=0.5)
ax.set_xlabel("air/water temperature (degrees Celsius)")
ax.set_ylabel("air density (kg m-3)")
ax.legend()
ax.grid()

# Albedo

In [ ]:
zen = np.linspace(0.0, 90.0, 100)
yday = np.linspace(0.0, 365.0, 101)
albedo = np.empty((yday.size, zen.size))


def plot(method, ax):
    for i, yd in enumerate(yday):
        pyairsea.albedo_water(method, zen.reshape(1, -1), yd, albedo[i : i + 1, ...])
    pc = ax.contourf(yday, zen, albedo.T, np.linspace(0.0, 1.0, 11))
    fig.colorbar(pc, ax=ax)
    ax.set_xlabel("year day")
    ax.set_ylabel("zenith angle (degrees)")
    ax.set_title(method.name)
    print("%s range: %s - %s" % (method.name, albedo.min(), albedo.max()))


fig, (ax1, ax2) = matplotlib.pyplot.subplots(ncols=2, figsize=(10, 4))
plot(pyairsea.AlbedoMethod.PAYNE, ax1)
plot(pyairsea.AlbedoMethod.COGLEY, ax2)